In [73]:
depth_tree = 7

In [74]:
""" Libraries """
import os
import json
import numpy as np
import pandas as pd
import psycopg2
from psycopg2.extensions import register_adapter, AsIs
from tqdm.auto import tqdm
import nbimporter
from Core import json_numpy_serializer

pd.set_option('display.float_format', lambda x: '%.6f' % x)
pd.set_option('display.max_rows', None)
tqdm.pandas(desc="Processing time")
register_adapter(np.int64, lambda val: AsIs(int(val)))

In [75]:
""" Parameters """
with open("para.json", "r") as json_file:
    para = json.load(json_file)
fold_data = para["fold_data"]

In [76]:
""" Data Concatenating """
files = os.listdir(fold_data)
data = []
for file in tqdm(files):
    df = pd.read_csv(filepath_or_buffer="{}/{}".format(fold_data, file), sep=",")
    df["taxiId"] = int(file.split(".")[0])
    data.append(df)

df = pd.concat(data)
print(len(df))

  0%|          | 0/1000 [00:00<?, ?it/s]

2102735


In [77]:
""" Dual Cell Filling """
indexing_keys_exist = set(df["indexing_key"].tolist())
indexing_keys_all = list(range(8 ** depth_tree))
indexing_keys_dual = [indexing_key for indexing_key in indexing_keys_all if indexing_key not in indexing_keys_exist]

df_dual = pd.DataFrame(indexing_keys_dual, columns=['indexing_key'])
df_dual['geometry'] = None
df_dual['state'] = None
df_dual['taxiId'] = None

In [78]:
""" File Writing """
df = pd.concat([df, df_dual])
df["distributing_key"] = df.apply(lambda x: int(x["indexing_key"] / 8), axis=1)
df.sort_values(by=['indexing_key'], inplace=True)
df.to_csv("data_{}.csv".format(depth_tree), header=False, index=False, mode='w')

In [79]:
""" Database Connection """
conn_params = "dbname='trajectory' user='postgres' host='localhost' password='990721' port=5432"
conn = psycopg2.connect(conn_params)
cur = conn.cursor()

In [80]:
""" Create Table """
create_table_sql = """
DROP TABLE IF EXISTS traj_{};
CREATE TABLE traj_{}
(
    taxiid BIGINT,
    indexing_key     INTEGER,
    state            SMALLINT,
    geometry         GEOMETRY(LINESTRINGZ),
    distributing_key INTEGER
);
""".format(depth_tree, depth_tree)
cur.execute(create_table_sql)
conn.commit()

""" Insert Data """
insert_data_sql = """
INSERT INTO traj_{} (taxiid, indexing_key,state, geometry, distributing_key)
VALUES (%s, %s, %s, st_geomfromtext(%s), %s);
""".format(depth_tree)

for index, row in tqdm(df.iterrows()):
    data = (row['taxiId'], row['indexing_key'], row['state'], row['geometry'], row['distributing_key'])
    cur.execute(insert_data_sql, data)
conn.commit()

0it [00:00, ?it/s]

In [81]:
""" Disconnect Database """
cur.close()
conn.close()